In [1]:
import tensorflow as tf
import numpy as np
import base64
import keras

In [36]:
# Compress images
import os
import sys
from PIL import Image

def resize(folder, fileName):
    filePath = os.path.join(folder, fileName)
    im = Image.open(filePath)
    w, h  = im.size
    newIm = im.resize((64, 64))
    newIm.save(filePath)

def bulkResize(imageFolder):
    imgExts = ["png", "jpg"]
    for path, dirs, files in os.walk(imageFolder):
        for fileName in files:
            ext = fileName[-3:].lower()
            if ext not in imgExts:
                continue
            resize(path, fileName)

imageFolder="/Users/Chrsitine/Desktop/Conjurers/images/yellow_oreos/" 
bulkResize(imageFolder)
print("Done resizing")

In [ ]:
# Load training data
x_train = []
y_train = []
for 

In [3]:
# Save trained model
model.save("trained_model1.hdf5")
model.save_weights("trained_weights1.hdf5")
print("Saved trained model and weights")